In [8]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

y_test = y_test - 769
y_train_valid = y_train_valid - 769
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

# For subject 1
X_train_valid_1 = X_train_valid[np.where(person_train_valid==0)[0]]
y_train_valid_1 = y_train_valid[np.where(person_train_valid==0)[0]]
X_test_1 = X_test[np.where(person_test==0)[0]]
y_test_1 = y_test[np.where(person_test==0)[0]]

class Dataset(torch.utils.data.Dataset):
    def __init__(self,X,Y):
        self.X = torch.FloatTensor(X)
        self.Y = torch.LongTensor(Y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]
    
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid,
                                                              test_size=0.2,shuffle=True)

train_set = Dataset(X_train,y_train)
val_set = Dataset(X_valid,y_valid)
test_set = Dataset(X_test, y_test)
print(X_train.shape)

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)
(1692, 22, 1000)


In [ ]:
for i in range(35,70):
    train_loader = torch.utils.data.DataLoader(train_set,batch_size=i,shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set,batch_size=20,shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_set,batch_size=40,shuffle=True)
    
    #目前这个最好
    #只有FC有dropout
    # [conv-relu]*2 -> 2*2 max-pooling -> [conv-relu]*3 -> 2*2 max_pooling -> (affine-relu)*2 -> affine -> softmax
    class Net(nn.Module):
        def __init__(self):
            super(Net,self).__init__()  # initial the model
            self.conv1 = nn.Conv1d(22,40,kernel_size = 2,stride = 2) 
            self.bn1 = nn.BatchNorm1d(40)
            self.conv2 = nn.Conv1d(40,60,kernel_size = 3,stride = 1) 
            self.bn2 = nn.BatchNorm1d(60) 
            self.pool1 = nn.MaxPool1d(2,2) 

            self.conv3 = nn.Conv1d(60,80,kernel_size = 3, stride = 1) 
            self.bn3 = nn.BatchNorm1d(80)
            self.conv4 = nn.Conv1d(80,100,kernel_size = 3, stride = 1) 
            self.bn4 = nn.BatchNorm1d(100)
            self.conv5 = nn.Conv1d(100,120,kernel_size = 3, stride = 2) #120*122
            self.bn5 = nn.BatchNorm1d(120)
            self.pool2 = nn.MaxPool1d(2,2) #120*61

            self.fc1 = nn.Linear(120*61, 300) # input dim , output dim
            self.bn6 = nn.BatchNorm1d(300)
            self.drop1 = nn.Dropout(0.8)
            self.fc2 = nn.Linear(300,40)  
            self.bn7 = nn.BatchNorm1d(40)
            self.drop2 = nn.Dropout(0.8)
            self.fc3 = nn.Linear(40,4)

        def forward(self,x):
            x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
            x = F.relu(self.bn3(self.conv3(x)))
            x = F.relu(self.bn4(self.conv4(x)))
            x = self.pool2(F.relu(self.bn5(self.conv5(x))))
            x = x.view(-1,120*61)

            x = self.drop1(F.relu(self.bn6(self.fc1(x))))
            x = self.drop2(F.relu(self.bn7(self.fc2(x))))
            x = self.fc3(x)
            return x
    net = Net()
    print(net)
    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.Adam(net.parameters(),lr = 0.01)
    optimizer = torch.optim.RMSprop(net.parameters(),lr = 0.001)
    print('batch size is ',i,'--------------------------------------------')
    val_acc_history = []
    train_acc_history = []
    for epoch in range(43):
        for i , data in enumerate(train_loader, 0):
            inputs, labels = data

            net.train()

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _,predicted = torch.max(outputs.data, 1)
            train_acc = (predicted == labels).sum().item() / len(labels)

            if i % 2 == 1:
                net.eval()
                val_correct, val_total = 0, 0 
                for val_data in val_loader:
                    val_images, val_labels = val_data
                    val_outputs = net(val_images)
                    _, val_predicted = torch.max(val_outputs.data, 1)
                    val_total += val_labels.size(0)
                    val_correct += (val_predicted == val_labels).sum().item()
                val_acc = val_correct / val_total
                print('Epoch {} | Iter{} | Loss{:.4f} | TrainAcc{:.4f} | val acc {:.4f}'.format(
                    epoch, i , loss, train_acc, val_acc))
                #writer.add_scalar('Train/Loss',loss,epoch*len(trainloader) + i)
                #writer.add_scalar('Train/ACC',train_acc,epoch*len(trainloader) + i)
                #writer.add_scalar('VAL/ACC',val_acc,epoch*len(trainloader) + i)
        net.eval()
        train_correct, train_total = 0, 0
        for train_data in train_loader:
            train_inputs, train_labels = train_data
            train_outputs = net(train_inputs)
            _, train_predicted = torch.max(train_outputs.data, 1)
            train_total += train_labels.size(0)
            train_correct += (train_predicted == train_labels).sum().item()
        train_acc = train_correct / train_total
        train_acc_history.append(train_acc)
        val_correct, val_total = 0, 0
        for val_data in val_loader:
            val_images, val_labels = val_data
            val_outputs = net(val_images)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()
        val_acc = val_correct / val_total
        val_acc_history.append(val_acc)
        #if val_acc == max(val_acc_history):
        #    net_best = Net()
        #    net_best.load_state_dict(net.state_dict())

    test_correct, test_total = 0, 0
    for test_data in test_loader:
        test_images, test_labels = test_data
        test_outputs = net(test_images)
        #test_outputs = net_best(test_images)
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_labels.size(0)
        test_correct += (test_predicted == test_labels).sum().item()
    test_acc = test_correct / test_total
    print('Test accuracy is: ',test_acc)
    plt.plot(train_acc_history,)
    plt.plot(val_acc_history)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy')
    plt.legend(['train_acc','val_acc'])

Net(
  (conv1): Conv1d(22, 40, kernel_size=(2,), stride=(2,))
  (bn1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(40, 60, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(60, 80, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(80, 100, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv1d(100, 120, kernel_size=(3,), stride=(2,))
  (bn5): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=7320, out_features=300, bias=True)
  (bn6): BatchNorm1d(300,

Epoch 4 | Iter23 | Loss1.3761 | TrainAcc0.2857 | val acc 0.4019
Epoch 4 | Iter25 | Loss1.3030 | TrainAcc0.3714 | val acc 0.3924
Epoch 4 | Iter27 | Loss1.4892 | TrainAcc0.2571 | val acc 0.3972
Epoch 4 | Iter29 | Loss1.4269 | TrainAcc0.3143 | val acc 0.3830
Epoch 4 | Iter31 | Loss1.2912 | TrainAcc0.4571 | val acc 0.3924
Epoch 4 | Iter33 | Loss1.2823 | TrainAcc0.4286 | val acc 0.4019
Epoch 4 | Iter35 | Loss1.3197 | TrainAcc0.2571 | val acc 0.3924
Epoch 4 | Iter37 | Loss1.4056 | TrainAcc0.2571 | val acc 0.4043
Epoch 4 | Iter39 | Loss1.3249 | TrainAcc0.4286 | val acc 0.4232
Epoch 4 | Iter41 | Loss1.4233 | TrainAcc0.3429 | val acc 0.3735
Epoch 4 | Iter43 | Loss1.3716 | TrainAcc0.2571 | val acc 0.3830
Epoch 4 | Iter45 | Loss1.2929 | TrainAcc0.3429 | val acc 0.3759
Epoch 4 | Iter47 | Loss1.3127 | TrainAcc0.3429 | val acc 0.3924
Epoch 5 | Iter1 | Loss1.3799 | TrainAcc0.4571 | val acc 0.3664
Epoch 5 | Iter3 | Loss1.3833 | TrainAcc0.3714 | val acc 0.4161
Epoch 5 | Iter5 | Loss1.3104 | TrainAcc0.1

Epoch 9 | Iter41 | Loss1.1467 | TrainAcc0.5429 | val acc 0.4917
Epoch 9 | Iter43 | Loss1.1006 | TrainAcc0.4000 | val acc 0.4468
Epoch 9 | Iter45 | Loss1.0878 | TrainAcc0.4286 | val acc 0.4586
Epoch 9 | Iter47 | Loss1.3298 | TrainAcc0.2857 | val acc 0.4397
Epoch 10 | Iter1 | Loss1.1851 | TrainAcc0.5429 | val acc 0.4326
Epoch 10 | Iter3 | Loss1.1027 | TrainAcc0.6000 | val acc 0.4350
Epoch 10 | Iter5 | Loss1.1826 | TrainAcc0.4286 | val acc 0.4279
Epoch 10 | Iter7 | Loss1.1527 | TrainAcc0.5429 | val acc 0.4870
Epoch 10 | Iter9 | Loss1.0311 | TrainAcc0.7714 | val acc 0.4681
Epoch 10 | Iter11 | Loss1.0507 | TrainAcc0.5429 | val acc 0.4704
Epoch 10 | Iter13 | Loss1.2757 | TrainAcc0.5143 | val acc 0.4444
Epoch 10 | Iter15 | Loss1.0369 | TrainAcc0.5143 | val acc 0.4326
Epoch 10 | Iter17 | Loss1.2308 | TrainAcc0.4286 | val acc 0.4728
Epoch 10 | Iter19 | Loss1.0723 | TrainAcc0.4571 | val acc 0.4657
Epoch 10 | Iter21 | Loss1.1000 | TrainAcc0.5429 | val acc 0.4421
Epoch 10 | Iter23 | Loss1.2979 | T

Epoch 15 | Iter7 | Loss0.8494 | TrainAcc0.6286 | val acc 0.4894
Epoch 15 | Iter9 | Loss0.9422 | TrainAcc0.6571 | val acc 0.4634
Epoch 15 | Iter11 | Loss0.8168 | TrainAcc0.6571 | val acc 0.4704
Epoch 15 | Iter13 | Loss0.9731 | TrainAcc0.6857 | val acc 0.4965
Epoch 15 | Iter15 | Loss0.8113 | TrainAcc0.7143 | val acc 0.5012
Epoch 15 | Iter17 | Loss0.8732 | TrainAcc0.6571 | val acc 0.4965
Epoch 15 | Iter19 | Loss1.0668 | TrainAcc0.5143 | val acc 0.4870
Epoch 15 | Iter21 | Loss1.0077 | TrainAcc0.4857 | val acc 0.4917
Epoch 15 | Iter23 | Loss0.9782 | TrainAcc0.5429 | val acc 0.5012
Epoch 15 | Iter25 | Loss0.9344 | TrainAcc0.6286 | val acc 0.5083
Epoch 15 | Iter27 | Loss0.7419 | TrainAcc0.7143 | val acc 0.4894
Epoch 15 | Iter29 | Loss0.9067 | TrainAcc0.6286 | val acc 0.4374
Epoch 15 | Iter31 | Loss0.9635 | TrainAcc0.6000 | val acc 0.4066
Epoch 15 | Iter33 | Loss0.8817 | TrainAcc0.5429 | val acc 0.4704
Epoch 15 | Iter35 | Loss0.7810 | TrainAcc0.7143 | val acc 0.5012
Epoch 15 | Iter37 | Loss0.8

Epoch 20 | Iter21 | Loss0.5645 | TrainAcc0.6571 | val acc 0.4846
Epoch 20 | Iter23 | Loss0.7618 | TrainAcc0.4857 | val acc 0.5035
Epoch 20 | Iter25 | Loss0.7809 | TrainAcc0.6286 | val acc 0.4492
Epoch 20 | Iter27 | Loss0.7923 | TrainAcc0.6571 | val acc 0.4988
Epoch 20 | Iter29 | Loss0.7499 | TrainAcc0.6857 | val acc 0.4894
Epoch 20 | Iter31 | Loss0.6751 | TrainAcc0.7143 | val acc 0.4704
Epoch 20 | Iter33 | Loss0.7793 | TrainAcc0.7714 | val acc 0.4704
Epoch 20 | Iter35 | Loss0.7841 | TrainAcc0.7143 | val acc 0.5106
Epoch 20 | Iter37 | Loss0.6646 | TrainAcc0.6857 | val acc 0.4775
Epoch 20 | Iter39 | Loss0.7567 | TrainAcc0.6571 | val acc 0.4846
Epoch 20 | Iter41 | Loss0.6423 | TrainAcc0.7429 | val acc 0.4846
Epoch 20 | Iter43 | Loss0.5999 | TrainAcc0.7429 | val acc 0.4563
Epoch 20 | Iter45 | Loss0.7006 | TrainAcc0.7429 | val acc 0.4468
Epoch 20 | Iter47 | Loss0.7622 | TrainAcc0.6000 | val acc 0.4468
Epoch 21 | Iter1 | Loss0.5487 | TrainAcc0.8286 | val acc 0.4894
Epoch 21 | Iter3 | Loss0.7

...

Epoch 17 | Iter1 | Loss0.7473 | TrainAcc0.6939 | val acc 0.4823
Epoch 17 | Iter3 | Loss0.8207 | TrainAcc0.6122 | val acc 0.4823
Epoch 17 | Iter5 | Loss0.7694 | TrainAcc0.6939 | val acc 0.4917
Epoch 17 | Iter7 | Loss0.6830 | TrainAcc0.7959 | val acc 0.4870
Epoch 17 | Iter9 | Loss0.7307 | TrainAcc0.7143 | val acc 0.4941
Epoch 17 | Iter11 | Loss0.9425 | TrainAcc0.5510 | val acc 0.4823
Epoch 17 | Iter13 | Loss0.8488 | TrainAcc0.6327 | val acc 0.4870
Epoch 17 | Iter15 | Loss0.7008 | TrainAcc0.7347 | val acc 0.5272
Epoch 17 | Iter17 | Loss0.8853 | TrainAcc0.6122 | val acc 0.4846
Epoch 17 | Iter19 | Loss0.7647 | TrainAcc0.6939 | val acc 0.4988
Epoch 17 | Iter21 | Loss0.7488 | TrainAcc0.7143 | val acc 0.5106
Epoch 17 | Iter23 | Loss0.8407 | TrainAcc0.6531 | val acc 0.5154
Epoch 17 | Iter25 | Loss0.8953 | TrainAcc0.5918 | val acc 0.4610
Epoch 17 | Iter27 | Loss0.7608 | TrainAcc0.6735 | val acc 0.4161
Epoch 17 | Iter29 | Loss0.6719 | TrainAcc0.7959 | val acc 0.5083
Epoch 17 | Iter31 | Loss0.8896

Epoch 24 | Iter17 | Loss0.4236 | TrainAcc0.8776 | val acc 0.5083
Epoch 24 | Iter19 | Loss0.5012 | TrainAcc0.7959 | val acc 0.4917
Epoch 24 | Iter21 | Loss0.4692 | TrainAcc0.8571 | val acc 0.4917
Epoch 24 | Iter23 | Loss0.5154 | TrainAcc0.8571 | val acc 0.5248
Epoch 24 | Iter25 | Loss0.6085 | TrainAcc0.7959 | val acc 0.4988
Epoch 24 | Iter27 | Loss0.3554 | TrainAcc0.8571 | val acc 0.4941
Epoch 24 | Iter29 | Loss0.5856 | TrainAcc0.7551 | val acc 0.5035
Epoch 24 | Iter31 | Loss0.3822 | TrainAcc0.8980 | val acc 0.5319
Epoch 24 | Iter33 | Loss0.6858 | TrainAcc0.6939 | val acc 0.5272
Epoch 25 | Iter1 | Loss0.5070 | TrainAcc0.8367 | val acc 0.5225
Epoch 25 | Iter3 | Loss0.5002 | TrainAcc0.8571 | val acc 0.5390
Epoch 25 | Iter5 | Loss0.4237 | TrainAcc0.8776 | val acc 0.5461
Epoch 25 | Iter7 | Loss0.6386 | TrainAcc0.7347 | val acc 0.5248
Epoch 25 | Iter9 | Loss0.4344 | TrainAcc0.8980 | val acc 0.5035
Epoch 25 | Iter11 | Loss0.4890 | TrainAcc0.8367 | val acc 0.5083
Epoch 25 | Iter13 | Loss0.4648

Epoch 31 | Iter33 | Loss0.4287 | TrainAcc0.7959 | val acc 0.4941
Epoch 32 | Iter1 | Loss0.3013 | TrainAcc0.8776 | val acc 0.4917
Epoch 32 | Iter3 | Loss0.3277 | TrainAcc0.8980 | val acc 0.4894
Epoch 32 | Iter5 | Loss0.3336 | TrainAcc0.8980 | val acc 0.5248
Epoch 32 | Iter7 | Loss0.4659 | TrainAcc0.8163 | val acc 0.5059
Epoch 32 | Iter9 | Loss0.5152 | TrainAcc0.8571 | val acc 0.5059
Epoch 32 | Iter11 | Loss0.3003 | TrainAcc0.9388 | val acc 0.5225
Epoch 32 | Iter13 | Loss0.4141 | TrainAcc0.8367 | val acc 0.5225
Epoch 32 | Iter15 | Loss0.3060 | TrainAcc0.8980 | val acc 0.4988
Epoch 32 | Iter17 | Loss0.3575 | TrainAcc0.8367 | val acc 0.5485
Epoch 32 | Iter19 | Loss0.4074 | TrainAcc0.8571 | val acc 0.5532
Epoch 32 | Iter21 | Loss0.4788 | TrainAcc0.9184 | val acc 0.5390
Epoch 32 | Iter23 | Loss0.3098 | TrainAcc0.8980 | val acc 0.5319
Epoch 32 | Iter25 | Loss0.3942 | TrainAcc0.8776 | val acc 0.5414
Epoch 32 | Iter27 | Loss0.4957 | TrainAcc0.7959 | val acc 0.5319
Epoch 32 | Iter29 | Loss0.5746

Epoch 39 | Iter15 | Loss0.3304 | TrainAcc0.8776 | val acc 0.4941
Epoch 39 | Iter17 | Loss0.2260 | TrainAcc0.9388 | val acc 0.4917
Epoch 39 | Iter19 | Loss0.2544 | TrainAcc0.9184 | val acc 0.4894
Epoch 39 | Iter21 | Loss0.3942 | TrainAcc0.8571 | val acc 0.5059
Epoch 39 | Iter23 | Loss0.2797 | TrainAcc0.8980 | val acc 0.4917
Epoch 39 | Iter25 | Loss0.2053 | TrainAcc0.9592 | val acc 0.4823
Epoch 39 | Iter27 | Loss0.1968 | TrainAcc0.9184 | val acc 0.5059
Epoch 39 | Iter29 | Loss0.3260 | TrainAcc0.8776 | val acc 0.4894
Epoch 39 | Iter31 | Loss0.3169 | TrainAcc0.9184 | val acc 0.4870
Epoch 39 | Iter33 | Loss0.2005 | TrainAcc0.9592 | val acc 0.4965
Epoch 40 | Iter1 | Loss0.2634 | TrainAcc0.9184 | val acc 0.4988
Epoch 40 | Iter3 | Loss0.2831 | TrainAcc0.8571 | val acc 0.4846
Epoch 40 | Iter5 | Loss0.1919 | TrainAcc0.9592 | val acc 0.4988
Epoch 40 | Iter7 | Loss0.3322 | TrainAcc0.8980 | val acc 0.4870
Epoch 40 | Iter9 | Loss0.4408 | TrainAcc0.8571 | val acc 0.4846
Epoch 40 | Iter11 | Loss0.4263

Epoch 2 | Iter21 | Loss1.3966 | TrainAcc0.3000 | val acc 0.3593
Epoch 2 | Iter23 | Loss1.3793 | TrainAcc0.3000 | val acc 0.3357
Epoch 2 | Iter25 | Loss1.3487 | TrainAcc0.2800 | val acc 0.3877
Epoch 2 | Iter27 | Loss1.3418 | TrainAcc0.3200 | val acc 0.3570
Epoch 2 | Iter29 | Loss1.3980 | TrainAcc0.2800 | val acc 0.3901
Epoch 2 | Iter31 | Loss1.3955 | TrainAcc0.3600 | val acc 0.4066
Epoch 2 | Iter33 | Loss1.4051 | TrainAcc0.4524 | val acc 0.3806
